## Wordle 5 set - solved using Graph
- Date : 22-Feb-2026
- Run Time < 1.25 Min (Local M4) (3 Mins on Colab)

## Find the sets of 5 words with no char in common

In [2]:
import requests
from tqdm.notebook import tqdm
from collections import defaultdict
from string import ascii_lowercase

### Get all words

In [4]:
def get_words():
    # https://gist.github.com/subhrm/5362767af06597bd1e216c59b760f6cb
    url="https://gist.githubusercontent.com/subhrm/5362767af06597bd1e216c59b760f6cb/raw/6bfa15d263d6d5b63840a8e5b64e04b382fdb079/valid-wordle-words.txt"
    resp = requests.get(url)
    print(f"{resp.status_code=}")
    resp.raise_for_status()
    word_list = [w for w in resp.text.split("\n") if len(w)==5]
    print(f"{len(word_list)=:,}")

    return word_list

word_list = get_words()

resp.status_code=200
len(word_list)=14,855


## Group Anagrams
> Convert words to numbers to keep track of anagrams

In [5]:
char_map = {c:i for i,c in enumerate(ascii_lowercase)}

def word_to_num(word):
    return sum( (1<<char_map[c]) for c in word)

# test
assert word_to_num("abc") == word_to_num("bac")

anagrams = defaultdict(list)
for w in word_list:
    if len(set(w)) == 5:
        # only keep words with 5 unique characters
        num = word_to_num(w)
        anagrams[num].append(w)

print(f"{len(anagrams)=:,}")

unique_list = sorted(anagrams.keys())

len(anagrams)=5,650


## Create graph 
-  the nodes are words 
- and there's a edge if the words share no common letter

In [8]:
g =  defaultdict(set)

for i,w1 in tqdm(enumerate(unique_list)):
    for j in range(i+1, len(unique_list)):
        w2 = unique_list[j]
        if (w1 & w2) == 0:
            g[w1].add(w2)

print(f"{len(g)=}")

0it [00:00, ?it/s]

len(g)=5418


# Solution: 1

Use the 5-clique solution. Iteratively find all complete sub-graphs with 5 nodes. Runs in `< 4 minutes`

In [9]:
solutions =  []

for w1 in tqdm(unique_list):
    f1 = g[w1]
    if len(f1) < 4:
        continue
    for w2 in f1:
        f2 = f1 & g[w2]
        if len(f2) < 3:
            continue
        for w3 in f2:
            f3 = f2 & g[w3]
            if len(f3) < 2:
                continue
            for w4 in f3:
                f4 = f3 & g[w4]
                for w5 in f4:
                    solutions.append((w1,w2,w3,w4,w5))

print(f"\n{len(solutions)=}")

  0%|          | 0/5650 [00:00<?, ?it/s]


len(solutions)=23


In [10]:
from itertools import product
for sol in solutions:
    for p in product(*[anagrams[w] for w in sol]):
        print(p)

('bling', 'treck', 'waqfs', 'jumpy', 'vozhd')
('joked', 'crumb', 'waqfs', 'phynx', 'glitz')
('pling', 'treck', 'waqfs', 'jumby', 'vozhd')
('dreck', 'jumbo', 'waqfs', 'phynx', 'glitz')
('brick', 'glent', 'waqfs', 'jumpy', 'vozhd')
('kreng', 'clipt', 'waqfs', 'jumby', 'vozhd')
('fjord', 'gucks', 'vibex', 'nymph', 'waltz')
('fjord', 'chunk', 'vibex', 'gymps', 'waltz')
('fjord', 'quack', 'wembs', 'phynx', 'glitz')
('brock', 'judge', 'waqfs', 'phynx', 'miltz')
('cromb', 'gived', 'waqfs', 'phynx', 'klutz')
('cromb', 'jived', 'waqfs', 'phynx', 'klutz')
('cromb', 'juked', 'waqfs', 'phynx', 'glitz')
('prick', 'glent', 'waqfs', 'jumby', 'vozhd')
('jacks', 'whump', 'blonx', 'gyved', 'fritz')
('glent', 'bruck', 'waqfs', 'jimpy', 'vozhd')
('kempt', 'brung', 'waqfs', 'cylix', 'vozhd')
('kempt', 'brung', 'waqfs', 'xylic', 'vozhd')
('blunk', 'waqfs', 'cimex', 'grypt', 'vozhd')
('clunk', 'waqfs', 'bemix', 'grypt', 'vozhd')
('bruck', 'moved', 'waqfs', 'phynx', 'glitz')
('bruck', 'goved', 'waqfs', 'phynx